In [19]:
import sys

import requests
from bs4 import BeautifulSoup

#import urllib
#from urllib.request import urlopen

import re
import pandas as pd

from matplotlib import pyplot as plt

%matplotlib inline

In [4]:
url_indeed = "https://www.indeed.com"
url_search="https://www.indeed.com/jobs?q=%22data+scientist%22&l=Philadelphia%2C+PA&radius=100"

#columns = "company location date_creat date_rev title salary jd url".split()
columns = "company location title salary jd url".split()
jobs = dict( (col,[]) for col in columns )

page = requests.get(url_search).text
soup = BeautifulSoup(page, 'lxml') # lxml or html.parser

regex = re.compile("Page 1 of ([\d,]+) jobs")
cnt = soup.find(id="searchCountPages").text
cnt = int( regex.findall(cnt)[0].replace(',', '') )

print( "Total number of jobs: {0}".format(cnt) )

#regex_date1 = re.compile("POT-Creation-Date:\s+(\d+-\d+-\d+)\s+")
#regex_date2 = re.compile("PO-Revision-Date:\s+(\d+-\d+-\d+)\s+")
regex_salary = re.compile('salaryText":"(.*)"')

for n in range(0, cnt, 10):
    url_next = url_search+'&start={0}'.format(n)
    print(n, end='..')
    page = requests.get(url_next).text
    soup = BeautifulSoup(page, 'lxml')
    
    all_matches = soup.find_all('div', attrs={'data-tn-component': 'organicJob'})
 
    ## has trouble, use the hard-coded ones
    #try:
    #    url_next = url_indeed + soup.find('link', attrs={'rel':['next']})['href']
    #except:
    #    url_next = None
        
    for m in all_matches:
               
        jd_url = m.find('a', attrs={'class':'jobtitle turnstileLink'.split()})
        jd_url = url_indeed + jd_url['href']
        # seems there are a lot of duplicated ones
        if jd_url is not None and jd_url in jobs['url']: continue
            
        jobs['url'].append( jd_url if jd_url else None) 
        
        company = m.find('span', attrs={"class":"company"})
        jobs['company'].append( company.text.strip() if company else None)
        
        location = m.find('span', attrs={"class":"location"})
        jobs['location'].append( location.text.strip() if location else None)
        
        title = m.find('a', attrs={'data-tn-element': "jobTitle"})
        jobs['title'].append( title.text.strip() if title else None)
    
        # many salaries are missing
        salary = m.find('span', attrs={'class': "salaryText"})
        jobs['salary'].append( salary.text.strip() if salary else None )
        
        jd_page = requests.get(jd_url).text
        jd_soup = BeautifulSoup(jd_page, 'lxml')
                   
        jd_desc = jd_soup.find('div',
                               attrs = {'id':["jobDescriptionText"],
                                        'class':["jobsearch-jobDescriptionText"]})
        jobs['jd'].append( jd_desc.text if jd_desc else None )

        # not the dates I want
        #date1 = regex_date1.findall(jd_page)[0]
        #date2 = regex_date2.findall(jd_page)[0]
        
        #if salary is None:
        #    try:    
        #        salary = regex_salary.findall(jd_page)[0]
        #    except:
        #        salary = None
        #else:
        #    salary = salary.text.strip()            
        #jobs['salary'].append( salary )
                
        #jobs['date_creat'].append(date1)
        #jobs['date_rev'].append(date2)
        
print(cnt)

jobs = pd.DataFrame(jobs, columns = columns)

# the duplicated ones has been filtered at the creation
#jobs = jobs.drop_duplicates()

Total number of jobs: 1453
0..10..20..30..40..50..60..70..80..90..100..110..120..130..140..150..160..170..180..190..200..210..220..230..240..250..260..270..280..290..300..310..320..330..340..350..360..370..380..390..400..410..420..430..440..450..460..470..480..490..500..510..520..530..540..550..560..570..580..590..600..610..620..630..640..650..660..670..680..690..700..710..720..730..740..750..760..770..780..790..800..810..820..830..840..850..860..870..880..890..900..910..920..930..940..950..960..970..980..990..1000..1010..1020..1030..1040..1050..1060..1070..1080..1090..1100..1110..1120..1130..1140..1150..1160..1170..1180..1190..1200..1210..1220..1230..1240..1250..1260..1270..1280..1290..1300..1310..1320..1330..1340..1350..1360..1370..1380..1390..1400..1410..1420..1430..1440..1450..1453


In [16]:
jobs.location = jobs.location.apply(lambda x: re.split('\d+',x)[0].strip())
jobs.describe()

,company,location,title,salary,jd,url
count,693,695,695,26,695,695
unique,425,98,497,23,688,695
top,Facebook,"New York, NY",Data Scientist,"$60,000 - $80,000 a year",The Applied AI and Machine Learning team at JP...,https://www.indeed.com/rc/clk?jk=ae30f9c4db21a...
freq,21,373,82,3,2,1


In [37]:
jobs.location.value_counts()[:20]
# go to New York!

New York, NY            373
Philadelphia, PA         57
Baltimore, MD            17
Princeton, NJ            15
Jersey City, NJ          14
Hoboken, NJ              10
Malvern, PA               9
Brooklyn, NY              9
Wilmington, DE            9
Spring House, PA          8
Woodcliff Lake, NJ        7
Piscataway, NJ            7
Parsippany, NJ            6
Manhattan, NY             6
Basking Ridge, NJ         6
Conshohocken, PA          5
Camden, NJ                5
Titusville, NJ            5
Long Island City, NY      5
Collegeville, PA          4
Name: location, dtype: int64

In [38]:
jobs[jobs.location=='Spring House, PA']
# Spring House = J&J
jobs[jobs.location=='Malvern, PA']
# Malvern = Vanguard
jobs[jobs.location=='Conshohocken, PA']
# RS Energy Group
jobs[jobs.location=='Collegeville, PA']
# GSK, IQVIA, IQVIA also have openning in Plymouth Meeting

,company,location,title,salary,jd,url
168,Johnson & Johnson Family of Companies,"Spring House, PA","Computational Immunologist, Immunology Data Sc...",None,Janssen Research & Development is a world-clas...,https://www.indeed.com/rc/clk?jk=b8946db7aad57...
335,Johnson & Johnson Family of Companies,"Spring House, PA","Scientist, Cell Line Development & Data Science",None,"Janssen Research & Development, LLC, a member ...",https://www.indeed.com/rc/clk?jk=39322d2edfaf2...
386,Johnson & Johnson Family of Companies,"Spring House, PA","Principal Scientist, Advanced Analytics - Onco...",None,"Janssen Research & Development, LLC., a divisi...",https://www.indeed.com/rc/clk?jk=58e12c68a825f...
396,Johnson & Johnson Family of Companies,"Spring House, PA","Data Science Co-Op, Integrated Clinical and Op...",None,"Caring for the world, one person at a time has...",https://www.indeed.com/rc/clk?jk=ff1979fb70531...
434,Johnson & Johnson Family of Companies,"Spring House, PA","POST-DOCTORAL FELLOW, MACHINE LEARNING FOR COM...",None,"Caring for the world, one person at a time has...",https://www.indeed.com/rc/clk?jk=c1062490ac38f...
610,Johnson & Johnson Family of Companies,"Spring House, PA","Post-Doctoral Researcher, Immunology Data Science",None,"Caring for the world, one person at a time has...",https://www.indeed.com/rc/clk?jk=03bb53475fe5a...
663,Johnson & Johnson Family of Companies,"Spring House, PA",Senior Statistician,None,Janssen Research & Development part of the Joh...,https://www.indeed.com/rc/clk?jk=f0e8120ef67db...
691,Johnson & Johnson Family of Companies,"Spring House, PA","Senior Scientist, Discovery Chemistry (1 of 2 ...",None,"Janssen Research & Development, L.L.C., a divi...",https://www.indeed.com/rc/clk?jk=c331a8c004b14...


,company,location,title,salary,jd,url
44,Vanguard,"Malvern, PA",FinTech Data Scientist,None,Low-cost investing has distinguished Vanguard ...,https://www.indeed.com/rc/clk?jk=4e4fd821658e2...
213,Vanguard,"Malvern, PA",Data Scientist,None,Capture the insights to stand for investors\nA...,https://www.indeed.com/rc/clk?jk=31612a5d94103...
232,Vanguard,"Malvern, PA",Data Scientist II,None,Collect the insights to stand for investors\nA...,https://www.indeed.com/rc/clk?jk=cb7a5587d7c5a...
244,Vanguard,"Malvern, PA",Principal Data Scientist,None,Do you want to lead and execute deep dive diag...,https://www.indeed.com/rc/clk?jk=0c99c4e42029d...
446,Vanguard,"Malvern, PA",Data Science Developer Advocate,None,Core Responsibilities:\nProvide in-person trai...,https://www.indeed.com/rc/clk?jk=77df8ce4b90eb...
509,Vanguard,"Malvern, PA",Senior ML Architect,None,"Duties and Responsibilities\nDefine, evaluate,...",https://www.indeed.com/rc/clk?jk=afac3d18b70a2...
566,Vanguard,"Malvern, PA",Machine Learning Engineer,None,Making data deliver for investors\nFrom our ra...,https://www.indeed.com/rc/clk?jk=961403ff17dde...
599,Vanguard,"Malvern, PA",Senior Marketing Data Analyst,None,"Analyzes data, assesses trends and develops ac...",https://www.indeed.com/rc/clk?jk=e9c2588f46f53...
678,Vanguard,"Malvern, PA",Experimentation Research Scientist,None,Do you want to work on an Advanced Experimenta...,https://www.indeed.com/rc/clk?jk=10bf1b63d6c9b...


,company,location,title,salary,jd,url
52,RS Energy Group,"Conshohocken, PA",Data Scientist,None,The Company\n\nRS Energy Group (RSEG) provides...,https://www.indeed.com/rc/clk?jk=f6c3ef6121272...
66,RS Energy Group,"Conshohocken, PA",Data Science Intern,None,The Company\nRS Energy Group (RSEG) provides i...,https://www.indeed.com/rc/clk?jk=a9b31e3557bdf...
283,RS Energy Group,"Conshohocken, PA",Software Developer Intern,None,The Company\n\nRS Energy Group (RSEG) provides...,https://www.indeed.com/rc/clk?jk=5e89d848d4014...
412,RS Energy Group,"Conshohocken, PA",Python Software Engineer,None,The Company\nRS Energy Group (RSEG) provides i...,https://www.indeed.com/rc/clk?jk=86515ccf09d20...
650,RS Energy Group,"Conshohocken, PA",Data Science QA Engineer,None,The Company\n\nRS Energy Group (RSEG) provides...,https://www.indeed.com/rc/clk?jk=9e648650f7362...


,company,location,title,salary,jd,url
86,GSK,"Collegeville, PA",Biological Data Scientist,None,Site Name: USA - Pennsylvania - Upper Providen...,https://www.indeed.com/rc/clk?jk=d9259555e743b...
201,GSK,"Collegeville, PA",Genomic Data Scientist,None,Site Name: USA - Pennsylvania - Upper Providen...,https://www.indeed.com/rc/clk?jk=72bd3bbbec1f3...
568,IQVIA,"Collegeville, PA",Assoc Data Science Director,None,IQVIA™ is the leading human data science compa...,https://www.indeed.com/rc/clk?jk=ec693d419d895...
668,GSK,"Collegeville, PA",Principal Statistician or Statistician - Resea...,None,Site Name: USA - Pennsylvania - Upper Providen...,https://www.indeed.com/rc/clk?jk=e2a57c00ef996...


In [35]:
jobs[jobs.location=='Philadelphia, PA'].groupby('company').count()

,location,title,salary,jd,url
company,,,,,
Accenture,1,1,0,1,1
"Amazon Web Services, Inc.",1,1,0,1,1
Aramark Food and Support Services Group Inc.,1,1,0,1,1
Benefits Data Trust,1,1,0,1,1
Bristol-Myers Squibb,1,1,0,1,1
Cadent,1,1,0,1,1
Cerner Corporation,1,1,0,1,1
Children's Hospital of Philadelphia,4,4,0,4,4
CitiusTech,1,1,0,1,1


In [53]:
jobs[jobs.location=='New York, NY'].loc[:,['company','location']].groupby('company').\
                            count().sort_values('location', ascending=False)

,location
company,
Facebook,21
JP Morgan Chase,14
"Amazon.com Services, Inc.",9
Spotify,7
"Two Sigma Investments, LLC.",6
...,...
Information Builders Inc.,1
Innovations for Poverty,1
Innovations for Poverty Action,1
